In [ ]:
# Initializing
from google.colab import drive
import re
import os
import math
import keras
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import pathlib
from keras.utils import np_utils
import cv2

drive.mount('/content/drive')
# Define constant value
RESEARCH_WORK_PATH = "/content/drive/My Drive/Colab Notebooks/BachelorResearch/"
MODEL_PATH = RESEARCH_WORK_PATH + "models/"
DATA_PATH = RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/unknown_spectro/"

Mounted at /content/drive


In [ ]:
def path_to_pic(dir_folder):
    list_of_pic = []
    for file in os.listdir(dir_folder):
        if file.endswith(".jpg"):
            directory = "%s%s" % (dir_folder, file)
            list_of_pic.append(directory)
    return list_of_pic

def make_unknown_dataset(path):
  x = []
  y = []
  for pic in path_to_pic(path):
    im = np.array(Image.open(pic))
    x.append(im)
    y.append(int(pathlib.Path(pic).stem[1])-1)
  x = np.array(x)
  x = x.astype("float32")
  x /= 255
  y = np.array(y)
  return x,y

In [ ]:
x_raw ,y = make_unknown_dataset(DATA_PATH)
y = np_utils.to_categorical(y, 4)
print(y.sum(axis=0))
for m in os.listdir(MODEL_PATH):
  if not "mel" in m:
    print("[INFO ] This is not a model that use melspectrogram. Pass")
    return
  print("[MODEL]", pathlib.Path(m).name)
  try:
    model = keras.models.load_model(MODEL_PATH + m)
  except OSError:
    print("[ERROR]This model seems broken.... (> <). Pass this Evaluation.")
    break
  s = model.input.shape
  print("input shape:", s)
  x = x_raw[:,:s[1],:]
  try:
    x = x.reshape(x.shape[0], s[1], s[2], s[3])
  except ValueError:
    x_rgb = []
    for i in x:
      tmp = cv2.cvtColor(i,cv2.COLOR_GRAY2RGB)
      # tmp = cv2.resize(tmp,(s[1],s[2]))
      x_rgb.append(tmp)
    x = np.array(x_rgb)
    x = x.reshape(x.shape[0], s[1], s[2], s[3])
  score = model.evaluate(x, y, verbose=2)
  print('accuracy:', score[1])
  print('loss:', score[0])
  print()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  app.launch_new_instance()


ValueError: ignored